In [1]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngest:
    root:Path
    source:str
    zipdir:Path
    unzipdir:Path

In [2]:
from ml_project.constants import *
from ml_project.utils.com import read_yml, create_dir

In [3]:
class ConfigurationManager:
    def __init__(
            self,
            conf = CONFIG_PATH,
            sche = SCHEMA_PATH,
            param = PARAMS_PATH):
        self.conf = read_yml(conf)
        self.sche = read_yml(sche)
        self.param = read_yml(param)

        create_dir([self.conf.artifact_root])

    def get_ingestion_conf(self) -> DataIngest:
        config = self.conf.data_ingestion

        create_dir([config.rootdir])

        data_ingest_config = DataIngest(
            root=config.rootdir,
            source=config.url,
            zipdir=config.localdir,
            unzipdir=config.unzipdir
        )

        return data_ingest_config


In [ ]:
import urllib
from ml_project import logger
import zipfile
import os

In [27]:
import urllib.request


class DataToIngested:
    def __init__(self, config: DataIngest):
        self.config=config

    def download(self):
        if not os.path.exists(self.config.zipdir):
                fname, fhead = urllib.request.urlretrieve(
                     url=self.config.source,
                     filename=self.config.zipdir
                )
                logger.info(f'the file {fname} has been downloaded into {self.config.zipdir}')
        else:
             logger.info(f'unable to download the file')

    def extract(self):
         unzip_path = self.config.unzipdir
         os.makedirs(unzip_path, exist_ok=True)
         
         with zipfile.ZipFile(self.config.zipdir, 'r') as z:
              z.extractall(unzip_path)
                

In [28]:
try:
    configuaration = ConfigurationManager()
    ingest_config = configuaration.get_ingestion_conf()
    data_ingestion = DataToIngested(config=ingest_config)
    data_ingestion.download()
    data_ingestion.extract()

except Exception as e:
    raise e

[2025-01-24 07:11:21,017 : INFO :com : loaded the yaml file from <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'>]
[2025-01-24 07:11:21,020 : INFO :com : loaded the yaml file from <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'>]
[2025-01-24 07:11:21,022 : INFO :com : loaded the yaml file from <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'>]
[2025-01-24 07:11:21,024 : INFO :com : created the file artifact]
[2025-01-24 07:11:21,026 : INFO :com : created the file artifacts/data_ingestion]
[2025-01-24 07:11:23,349 : INFO :915750208 : the file artifacts/data_ingestion/data.zip has been downloaded into artifacts/data_ingestion/data.zip]
